In [1]:
import networkx as nx

In [2]:
from github import Github

# XXX: Specify your own access token here

ACCESS_TOKEN = '31ecdab1c572a467b88b19d670e09aed9c431fa4'

# Specify a username and repository of interest for that user.

USER = 'Dumbris'
REPO = 'rexpro-erl'

client = Github(ACCESS_TOKEN, per_page=100)
user = client.get_user(USER)
repo = user.get_repo(REPO)

# Get a list of people who have bookmarked the repo.
# Since you'll get a lazy iterator back, you have to traverse
# it if you want to get the total number of stargazers.

stargazers = [ s for s in repo.get_stargazers() ]
print "Number of stargazers", len(stargazers)

g = nx.DiGraph()
g.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)

for sg in stargazers:
    g.add_node(sg.login + '(user)', type='user')
    g.add_edge(sg.login + '(user)', repo.name + '(repo)', type='gazes')
    
    

Number of stargazers 3


In [12]:
REPOS = [
    "vhf/free-programming-books",
    "jquery/jquery"
]

user.get_events
contribs = repo.get_contributors()
contribs


<github.PaginatedList.PaginatedList instance at 0x7fa5bf480b48>

In [28]:
reps = user.get_repos()

In [19]:
evs = user.get_events()

In [25]:
help(evs.get_page)

Help on method get_page in module github.PaginatedList:

get_page(self, page) method of github.PaginatedList.PaginatedList instance



In [34]:
contribs = {}
for _repo in reps[:3]:
    users = []
    for _user in _repo.get_contributors():
        users.append(_user.login)
    contribs[_repo.name] = users
    
contribs    

{u'ansible-tsung': [u'rodo'],
 u'blueprints-scala': [u'vikraman', u'anvie', u'aembleton'],
 u'blueprints-scala-example': [u'Dumbris']}

In [3]:
# Let's add each stargazer's additional starred repos and add edges 
# to find additional interests.

MAX_REPOS = 100

for i, sg in enumerate(stargazers):
    print sg.login
    try:
        for starred in sg.get_starred()[:MAX_REPOS]: # Slice to avoid supernodes
            g.add_node(starred.name + '(repo)', type='repo', lang=starred.language, \
                       owner=starred.owner.login)
            g.add_edge(sg.login + '(user)', starred.name + '(repo)', type='gazes')
    except Exception, e: #ssl.SSLError:
        print "Encountered an error fetching starred repos for", sg.login, "Skipping."

    print "Processed", i+1, "stargazers' starred repos"
    print "Num nodes/edges in graph", g.number_of_nodes(), "/", g.number_of_edges()
    print "Rate limit", client.rate_limiting

Dumbris
Processed 1 stargazers' starred repos
Num nodes/edges in graph 102 / 101
Rate limit (4996, 5000)
jfenton
Processed 2 stargazers' starred repos
Num nodes/edges in graph 197 / 198
Rate limit (4995, 5000)
Papipo
Processed 3 stargazers' starred repos
Num nodes/edges in graph 296 / 297
Rate limit (4994, 5000)


In [4]:
# Save your work by serializing out another snapshot of the graph
nx.write_gpickle(g, "github_my.gpickle.1")
#

In [7]:
mtsw_users = [n for n in g if g.node[n]['type'] == 'user']
h = g.subgraph(mtsw_users)

nx.write_graphml(g, "github_my.graphml")

NetworkXError: GraphML writer does not support <type 'NoneType'> as data values.

In [10]:
import os
import json
from IPython.display import IFrame
from IPython.core.display import display
from networkx.readwrite import json_graph

print "Stats on the full graph"
print nx.info(g)
print

# Create a subgraph from a collection of nodes. In this case, the
# collection is all of the users in the original interest graph

mtsw_users = [n for n in g if g.node[n]['type'] == 'repo']
h = g.subgraph(mtsw_users)

print "Stats on the extracted subgraph"
print nx.info(h)

# Visualize the social network of all people from the original interest graph.

d = json_graph.node_link_data(g)
json.dump(d, open('resources/ch07-github/force.json', 'w'))


# IPython Notebook can serve files and display them into
# inline frames. Prepend the path with the 'files' prefix.

# A D3 template for displaying the graph data.
viz_file = 'files/resources/ch07-github/force.html'

# Display the D3 visualization.

display(IFrame(viz_file, '100%', '600px'))

Stats on the full graph
Name: 
Type: DiGraph
Number of nodes: 296
Number of edges: 297
Average in degree:   1.0034
Average out degree:   1.0034

Stats on the extracted subgraph
Name: 
Type: DiGraph
Number of nodes: 293
Number of edges: 0
Average in degree:   0.0000
Average out degree:   0.0000
